### Imports and installs

In [1]:
!pip install esm
!pip install omegaconf
!pip install torch
!pip install huggingface-hub

  Using cached attrs-24.2.0-py3-none-any.whl.metadata (11 kB)
  Using cached fsspec-2024.9.0-py3-none-any.whl.metadata (11 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
  Using cached huggingface_hub-0.26.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 17.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 MB 17.1 MB/s eta 0:00:0000:0100:01
Using cached sympy-1.13.1-py3-none-any.whl (6.2 MB)
Using cached attrs-24.2.0-py3-none-any.whl (63 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 11.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 873.1/873.1 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━

In [2]:
import torch
import torch.nn.functional as F
import torch.nn as nn

from model import DNADecoder

from omegaconf import OmegaConf
from torch.nn import TransformerDecoderLayer, TransformerDecoder
from transformers import AutoModel, AutoTokenizer

/Users/rishabjain/Desktop/Research/shriya/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Set up models

In [4]:
config = OmegaConf.load("config.yaml")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
choformer = DNADecoder(config).to(device).eval()
choformer.load_state_dict(torch.load("best_model.pth", map_location=device))

esm_model = AutoModel.from_pretrained("facebook/esm2_t6_8M_UR50D").to(device)
esm_tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t6_8M_UR50D")

/var/folders/ff/pbhg09jd53nb6lhsv863ym9c0000gn/T/ipykernel_75791/2697218582.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  choformer.load_state_dict(torch.load("best_mo

In [9]:
# import protein sequences
import pandas as pd
proteins = "/Users/rishabjain/Desktop/Research/choformer/inference/cho_low_exp_prot.csv"

df = pd.read_csv(proteins)

# convert column protein to list of sequences
sequences = df["protein"].tolist()

### Inference

In [12]:
protein_sequences = sequences[:10]
longest_protein_length = max([len(sequence) for sequence in protein_sequences])

protein_tokens = esm_tokenizer(
    protein_sequences,
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=longest_protein_length
).to(device)


with torch.no_grad():
    protein_embeddings = esm_model(**protein_tokens).last_hidden_state.squeeze(0)

In [13]:
outputs = choformer.generate(protein_embeddings)
print("GENERATED DNA SEQUENCES:", outputs['generated_sequences'])

GENERATED DNA SEQUENCES: ['ATGTCCTGGACCTCTGCCGACCCCGGGCAGAGCTCCCGCCCCCGCTGCCGCTTCCTGGAGGCCTCCCGCCTGGAGCGCCCCGTGCAGCTGGCCTGCTCCCGCCTGGAGCGCCGCTTCCGCGTGGAGCCCCGCGTCACCTTCCCGCCCGCCGAGCTCTGCTGCCGCCAGCGCCGCCGCCGCCGCGCGGCCTGCGGCCGCCGCGGCCTGGGCTGCGGCGCCCAGCTCAGCGCCAGCCCCGGCACCGGCGGCGTCTTCGGGGCCCCGAGCCCCGGGCGCCGCGCCGGCGCCGGGCCCGGCGGCCGCGCGGCCGGCGCCTGGCCGCCGTGCGCCGCGCTGCTGGTGGGCCGCGGCGAGGACGGCCCCGCCCGGGCCACCCTGGGCGCCGGCACCTCCCGCCCCCGGGACCCCGGGCCCTGGACCGCCGCCCGCCCCTGGCACGTGCGCCGCAGCCTGGAGGCCGCCTGGCCGCGCCTGGCCGGCCTGGCCCCGGCGGAGGCCGGCTGCGCCCGCCGCCGCGTCTGCCCGCGCGTCTCCCCCTCCCCCTTCTGCGGCAGCAACGCCTGCCTGCCCGGCGGCGCCCGGCAGCAGGTGCCCGGGGTCTGCGCCGCCCGGCAGCTCTCCTCCTCCAGCCAGCGCCGCAGCGAGGCCCTGGCCGGGGCCCCCCTGGACAATGCCCCCAAGGAGTACCCCCCCAAGATCCAGCAGCTGGTGCAGGACATTGCCAGCCTGACCCTGCTGGAGATCAGTGACCTGAATGAGCTGCTGAAGAAGACCCTGAAGATCCAGGATGTGGGCCTGATGCCCATGGGGGGCATGATGCCTGGGGCTGTGCCTGCCGCCGCCGCCGCCGCCCCCGAGGTGGCTGAGGGGGAGGACATCCCCAAGCAGAAGGAGCGCACCCACTTTACAGTGCGCCTGACAGAGGCCAAGCCTGTGGACAAGGTGAAGCTGATCAAGGAGATCAAGAATTATG